In [ ]:
import numpy as np
import pandas as pd

# df = pd.read_csv('movie_data_merged.csv')

In [ ]:
# df.to_csv(r'backup_merged.csv', index=False)

In [ ]:
df = pd.read_csv('backup_merged.csv')

df.head()

In [ ]:
df.director_cn = df.director_cn.str.strip()


In [ ]:
df[df.director_cn == 'United States of America']


In [ ]:
# df.groupby(by='director').director_cn.value_counts()

In [ ]:
df = df[df.director_cn != 'United States of America']
# df1.head(50)
df

In [ ]:
df1 = df.drop(columns=['critic_name', 'critic_cn', 'critic_org', 'movie_name', 'movie_year', 'href', 'director_birthYr'])


In [ ]:
df1

In [ ]:
df1 = df.groupby('director_cn')['director'].value_counts().reset_index(name='count')
df1

In [ ]:
df1["string"] = df1["director"]+ ": " +df1["count"].map(str) + np.where(df1["count"] >1, ' votes', ' vote')
df1["string"]

In [ ]:
# df1.sort_values(by="count").tail(30)

In [ ]:
output = df1.groupby('director_cn')['string'].apply(lambda x: "<div id='director'><h1><b>Ratings of directors born outside the US</b></h1><P>%s</P></div>" % '</p><p> '.join(x)).reset_index(name='properties.article')
output
                                                    
                                                    
                                                    

In [ ]:
output.iloc[3]['properties.article']

In [ ]:
dirs = df1.groupby('director_cn')['director'].nunique().reset_index(name='properties.headline')
dirs

In [ ]:
output = output.merge(dirs, how='left', on='director_cn')

In [ ]:
output

In [ ]:
output['properties.headline'] = output['properties.headline'].map(str) + np.where(output['properties.headline'] >1, " directors", " director")



In [ ]:
output

In [ ]:
output['properties.color'] = "#ff6200"

In [ ]:
output

In [ ]:

import requests
import json
from pandas import json_normalize

In [ ]:
with open('custom.geo.json') as json_data:
    geometry_data = json.load(json_data)

In [ ]:
df2 = pd.DataFrame.from_dict(json_normalize(geometry_data['features']),orient='columns')




In [ ]:
df2

In [ ]:
output['properties.country'] = output['director_cn']
output

In [ ]:
df_new = pd.merge(output, df2, how="left", left_on="properties.country", right_on="properties.sovereignt")


In [ ]:

df_new

In [ ]:
df_new = df_new.dropna(subset=['geometry.coordinates'])

In [ ]:
my_json = json.loads(df_new.to_json(orient='records'))


In [ ]:
my_json


In [ ]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data


In [ ]:
geo_format = process_to_geojson(my_json)

In [ ]:
geo_format

In [ ]:
with open('geo-data12-3.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('geo-data12-3.js', 'a') as outfile:
    json.dump(geo_format, outfile)